In [2]:
import pandas as pd
import json
from requests import get
from bs4 import BeautifulSoup
import json
import numpy as np
import time
import re

In [4]:
#api url and info
base_url = 'https://c3po.crossfit.com/api/leaderboards/v2/competitions'
competition = 'open'
year = 2023 #used 2023 to 2020
gender = 1 # genders 1 and 2 (male and female)

#store score display  and rank columns
score_columns_display = {}  
score_columns_rank = {} 

#list to store data from all pages and genders
all_data = []  

page = 1
while True:
    url = f"{base_url}/{competition}/{year}/leaderboards?division={gender}&page={page}"
    response = get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    json_data = soup.text
    data = json.loads(json_data)
    leaderboard_rows = data.get("leaderboardRows", [])

    if not leaderboard_rows:  # Break if no more data available
        break

    for row in leaderboard_rows:
        entrant = row.get("entrant", {})
        scores = row.get("scores", {})

        data_dict = {
            "competitorId": entrant.get("competitorId"),
            "competitorName": entrant.get("competitorName"),
            "firstName": entrant.get("firstName"),
            "lastName": entrant.get("lastName"),
            "status": entrant.get("status"),
            "gender": entrant.get("gender"),
            "age": entrant.get("age"),
            "countryOfOriginCode": entrant.get("countryOfOriginCode"),
            "countryOfOriginName": entrant.get("countryOfOriginName"),
            "regionId": entrant.get("regionId"),
            "regionName": entrant.get("regionName"),
            "affiliateId": entrant.get("affiliateId"),
            "affiliateName": entrant.get("affiliateName"),
            "height": entrant.get("height"),
            "weight": entrant.get("weight"),
            "overallRank": row.get("overallRank"),
            "overallScore": row.get("overallScore"),
        }

        for score in scores:
            ordinal = score.get("ordinal")
            score_display_column = f"score_display_{ordinal}"
            rank_column = f"rank_{ordinal}"

            data_dict[score_display_column] = score.get("scoreDisplay")
            data_dict[rank_column] = score.get("rank")

            if score_display_column not in score_columns_display:
                score_columns_display[score_display_column] = []
            score_columns_display[score_display_column].append(score.get("scoreDisplay"))

            if rank_column not in score_columns_rank:
                score_columns_rank[rank_column] = []
            score_columns_rank[rank_column].append(score.get("rank"))

        all_data.append(data_dict)

    page += 1

# Add score display and rank columns to the data dictionary
for col, values in score_columns_display.items():
    for i, data_dict in enumerate(all_data):
        data_dict[col] = values[i]

for col, values in score_columns_rank.items():
    for i, data_dict in enumerate(all_data):
        data_dict[col] = values[i]

# Create DataFrame
df = pd.DataFrame(all_data)

# Save the data to a CSV file
df.to_csv(f'../data/raw/open_{year}_gender_{gender}.csv', index=False, encoding='utf-8')

# Display the DataFrame
display(df)

,competitorId,competitorName,firstName,lastName,status,gender,age,countryOfOriginCode,countryOfOriginName,regionId,...,overallRank,overallScore,score_display_1,rank_1,score_display_2,rank_2,score_display_3,rank_3,score_display_4,rank_4
0,469656,Jeffrey Adler,Jeffrey,Adler,ACT,M,29,CA,Canada,35,...,1,107,298 reps,6,181 reps,37,312 lbs,59,8:01,5
1,300638,Tola Morakinyo,Tola,Morakinyo,ACT,M,27,US,United States,35,...,2,219,277 reps,64,175 reps,144,327 lbs,10,7:44,1
2,676693,Colten Mertens,Colten,Mertens,ACT,M,25,US,United States,34,...,3,367,274 reps,88,180 reps,49,302 lbs,174,9:29,56
3,663689,Tyler Christophel,Tyler,Christophel,ACT,M,33,US,United States,35,...,4,375,278 reps,61,172 reps,261,317 lbs,26,8:55,27
4,1031875,Roldan Goldbaum,Roldan,Goldbaum,ACT,M,23,MX,Mexico,34,...,5,526,273 reps,104,170 reps,359,317 lbs,27,9:10,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169444,374553,Liang Kong,Liang,Kong,ACT,M,41,CN,China,28,...,157742,582777,,152041,,148203,,148144,,134389
169445,27287,Matteo Piran,Matteo,Piran,ACT,M,43,IT,Italy,29,...,157742,582777,,152041,,148203,,148144,,134389
169446,25467,Robert Cummings,Robert,Cummings,ACT,M,51,US,United States,35,...,157742,582777,,152041,,148203,,148144,,134389
169447,1201495,Pedro Pereira,Pedro,Pereira,ACT,M,34,PT,Portugal,29,...,157742,582777,,152041,,148203,,148144,,134389
